# Welcome to the SAHI detection notebook. 

This notebook introduces you to running the sliced detection method (SAHI).  This is a lightweight wrapper on the SAHI method that does some pre-processing on the image before detection.  Results are returned to a pandas dataframe for further inspection and saved to the output path specified in a CSV formatted output.  More features are captured in the command line interface, such as creating a 

# First, import needed dependencies

In [1]:
import sys
sys.path.append('..')
from pathlib import Path
from huggingface_hub import hf_hub_download
from pathlib import Path
from sdcat.detect.sahi_detector import run_sahi_detect
from sahi import AutoDetectionModel

# Run detection

Let's process at 50 percent reduced size since the raw image is very large 7952x5304 and we don't need the full resolution to get reasonable results.

In [2]:
scale_percent = 50
image_path = Path.cwd().parent / 'sdcat' / 'tests' / 'data' / 'bird' / 'DSC00770.JPG'
csv_out_path =  Path.cwd() / 'sahi' / f'{image_path.stem}.csv'
slice_width = 1600
slice_height = 1600
confidence = 0.1

# Download the MBARI UAV model
model_path = hf_hub_download(repo_id="MBARI-org/uav-yolov5", filename="best.pt")
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=model_path,
    config_path=model_path,
    confidence_threshold=confidence,
    device='cpu',
)

# Create the output directory 
csv_out_path.parent.mkdir(parents=True, exist_ok=True)

# Run the detection
df_detections = run_sahi_detect(scale_percent=scale_percent, 
                                image_path=image_path,
                                csv_out_path=csv_out_path,
                                slice_width=slice_width,
                                slice_height=slice_height,
                                detection_model=detection_model)

02/27/2024 10:08:17 - INFO - sdcat -   Processing /Users/dcline/Dropbox/code/sdcat/sdcat/tests/data/bird/DSC00770.JPG
02/27/2024 10:08:17 - INFO - sdcat -   Using slice size width: 1600 and height: 1600


Performing prediction on 12 number of slices.


02/27/2024 10:08:26 - INFO - sdcat -   ObjectPrediction<
    bbox: BoundingBox: <(1652.4223022460938, 1269.49462890625, 1699.4335327148438, 1308.540771484375), w: 47.01123046875, h: 39.046142578125>,
    mask: None,
    score: PredictionScore: <value: 0.27579379081726074>,
    category: Category: <id: 14, name: Seagull>>
02/27/2024 10:08:26 - INFO - sdcat -   ObjectPrediction<
    bbox: BoundingBox: <(1616.7871704101562, 1135.657943725586, 1669.92333984375, 1169.05517578125), w: 53.13616943359375, h: 33.39723205566406>,
    mask: None,
    score: PredictionScore: <value: 0.24441100656986237>,
    category: Category: <id: 14, name: Seagull>>
02/27/2024 10:08:26 - INFO - sdcat -   ObjectPrediction<
    bbox: BoundingBox: <(914.9107, 2115.8342, 939.04364, 2148.281), w: 24.1329345703125, h: 32.44677734375>,
    mask: None,
    score: PredictionScore: <value: 0.22180786728858948>,
    category: Category: <id: 14, name: Seagull>>
02/27/2024 10:08:26 - INFO - sdcat -   ObjectPrediction<
    b

# Results

Results are saved to the **csv_out_path**. Let's take a look at the first few rows of the dataframe.

In [3]:
df_detections.head()

,image_path,class,score,area,saliency,...,y,xx,xy,w,h
0,/Users/dcline/Dropbox/code/sdcat/sdcat/tests/d...,Seagull,0.275794,1835.607208,-1,...,2538.989258,3398.867065,2617.081543,94.022461,78.092285
0,/Users/dcline/Dropbox/code/sdcat/sdcat/tests/d...,Seagull,0.244411,1774.600981,-1,...,2271.315887,3339.846680,2338.110352,106.272339,66.794464
0,/Users/dcline/Dropbox/code/sdcat/sdcat/tests/d...,Seagull,0.221808,783.035950,-1,...,4231.668457,1878.087280,4296.562012,48.265869,64.893555
0,/Users/dcline/Dropbox/code/sdcat/sdcat/tests/d...,Seagull,0.196877,1225.281864,-1,...,3244.703613,2511.534668,3298.583984,90.963135,53.880371
0,/Users/dcline/Dropbox/code/sdcat/sdcat/tests/d...,Seagull,0.183748,85.334829,-1,...,2197.486328,4237.900421,2215.936584,18.500519,18.450256


In [4]:
!ls -l /Users/dcline/Dropbox/code/sdcat/examples/sahi/

total 8
-rw-r--r--  1 dcline  513  2032 Feb 27 10:08 DSC00770.csv
